#  Predict RFC (Request for change) and possible failure misconfiguration of ITSM assets.

## Import necessary packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
%matplotlib inline
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder,scale
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report,f1_score,recall_score
import datetime as dt
from scipy import stats
import pandas.util.testing as tm

C:\Users\DELL\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Load the Dataset

In [3]:
data_parser=lambda c: pd.to_Dataframe(c,format='%d/%m/%Y %H:%M:%s')
data=pd.read_csv('C:\\Users\DELL\Desktop\Rubixe projects\Mar2020\ITSM_data.csv', parse_dates=['Open_Time','Reopen_Time','Close_Time','Resolved_Time'])

In [4]:
data.head()

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4.0,...,NaT,2013-04-11 13:50:00,2013-04-11 13:51:00,"3,87,16,91,111",Other,1.0,SD0000007,2.0,NaN,NaN
1,1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3.0,...,2013-02-12 12:31:00,2013-02-12 12:36:00,2013-02-12 12:36:00,"4,35,47,86,389",Software,1.0,SD0000011,1.0,NaN,NaN
2,2,DTA000024,application,Desktop Application,WBS000092,IM0000006,Closed,NS,3,NaN,...,NaT,2014-01-13 15:12:00,2014-01-13 15:13:00,"4,84,31,19,444",No error - works as designed,1.0,SD0000017,NaN,NaN,NaN
3,3,WBA000124,application,Web Based Application,WBS000088,IM0000011,Closed,4,4,4.0,...,NaT,2013-11-14 09:31:00,2013-11-14 09:31:00,"4,32,18,33,333",Operator error,1.0,SD0000025,NaN,NaN,NaN
4,4,WBA000124,application,Web Based Application,WBS000088,IM0000012,Closed,4,4,4.0,...,NaT,2013-08-11 13:55:00,2013-08-11 13:55:00,"3,38,39,03,333",Other,1.0,SD0000029,NaN,NaN,NaN


In [5]:
data.shape

(46606, 26)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46606 entries, 0 to 46605
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  46606 non-null  int64         
 1   CI_Name                     46606 non-null  object        
 2   CI_Cat                      46495 non-null  object        
 3   CI_Subcat                   46495 non-null  object        
 4   WBS                         46606 non-null  object        
 5   Incident_ID                 46606 non-null  object        
 6   Status                      46606 non-null  object        
 7   Impact                      46606 non-null  object        
 8   Urgency                     46606 non-null  object        
 9   Priority                    45226 non-null  float64       
 10  number_cnt                  46606 non-null  float64       
 11  Category                    46606 non-null  object    

In [7]:
data.describe()

,Unnamed: 0,Priority,number_cnt,No_of_Reassignments,No_of_Related_Interactions,No_of_Related_Incidents,No_of_Related_Changes
count,46606.00000,45226.000000,46606.000000,46605.000000,46492.000000,1222.000000,560.000000
mean,23302.50000,4.215805,0.499658,1.131831,1.149897,1.669394,1.058929
std,13454.13766,0.705624,0.288634,2.269774,2.556338,3.339687,0.403596
min,0.00000,1.000000,0.000023,0.000000,1.000000,1.000000,1.000000
25%,11651.25000,4.000000,0.248213,0.000000,1.000000,1.000000,1.000000
50%,23302.50000,4.000000,0.500269,0.000000,1.000000,1.000000,1.000000
75%,34953.75000,5.000000,0.749094,2.000000,1.000000,1.000000,1.000000
max,46605.00000,5.000000,0.999997,46.000000,370.000000,63.000000,9.000000


## Checking for EDA

In [8]:
data=data.drop(['Urgency','Impact','Alert_Status','No_of_Related_Incidents','Status','Open_Time','Reopen_Time','Close_Time','Resolved_Time'],axis=1)

In [9]:
data=data.iloc[:,:]
data.head()

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,4.0,0.601292,incident,KM0000553,26.0,"3,87,16,91,111",Other,1.0,SD0000007,NaN,NaN
1,1,WBA000124,application,Web Based Application,WBS000088,IM0000005,3.0,0.415050,incident,KM0000611,33.0,"4,35,47,86,389",Software,1.0,SD0000011,NaN,NaN
2,2,DTA000024,application,Desktop Application,WBS000092,IM0000006,NaN,0.517551,request for information,KM0000339,3.0,"4,84,31,19,444",No error - works as designed,1.0,SD0000017,NaN,NaN
3,3,WBA000124,application,Web Based Application,WBS000088,IM0000011,4.0,0.642927,incident,KM0000611,13.0,"4,32,18,33,333",Operator error,1.0,SD0000025,NaN,NaN
4,4,WBA000124,application,Web Based Application,WBS000088,IM0000012,4.0,0.345258,incident,KM0000611,2.0,"3,38,39,03,333",Other,1.0,SD0000029,NaN,NaN


In [10]:
data.shape

(46606, 17)

In [11]:
data.isna().sum().to_frame().T

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,0,111,111,0,0,1380,0,0,0,1,1,460,114,0,46046,46046


In [12]:
data['No_of_Related_Changes'].fillna(method='ffill',inplace=True)

In [13]:
data.isna().sum().to_frame().T

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,0,111,111,0,0,1380,0,0,0,1,1,460,114,0,10,46046


In [14]:
data['No_of_Related_Changes'].fillna(method='bfill',inplace=True)

In [15]:
data.isna().sum().to_frame().T

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,0,111,111,0,0,1380,0,0,0,1,1,460,114,0,0,46046


In [16]:
data['Related_Change'].fillna(method='ffill',inplace=True)

In [17]:
data.isna().sum().to_frame().T

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,0,111,111,0,0,1380,0,0,0,1,1,460,114,0,0,10


In [18]:
data['Related_Change'].fillna(method='bfill',inplace=True)

In [19]:
data.isna().sum().to_frame().T

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,0,111,111,0,0,1380,0,0,0,1,1,460,114,0,0,0


In [20]:
data.shape

(46606, 17)

In [21]:
data.dropna(inplace=True,how='any')

In [22]:
data.shape

(44554, 17)

In [23]:
data.head()

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,4.0,0.601292,incident,KM0000553,26.0,"3,87,16,91,111",Other,1.0,SD0000007,1.0,C00000056
1,1,WBA000124,application,Web Based Application,WBS000088,IM0000005,3.0,0.415050,incident,KM0000611,33.0,"4,35,47,86,389",Software,1.0,SD0000011,1.0,C00000056
3,3,WBA000124,application,Web Based Application,WBS000088,IM0000011,4.0,0.642927,incident,KM0000611,13.0,"4,32,18,33,333",Operator error,1.0,SD0000025,1.0,C00000056
4,4,WBA000124,application,Web Based Application,WBS000088,IM0000012,4.0,0.345258,incident,KM0000611,2.0,"3,38,39,03,333",Other,1.0,SD0000029,1.0,C00000056
5,5,WBA000124,application,Web Based Application,WBS000088,IM0000013,4.0,0.006676,incident,KM0000611,4.0,"3,38,34,36,944",Other,1.0,SD0000031,1.0,C00000056


In [24]:
data.isna().sum().to_frame().T

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44554 entries, 0 to 46605
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  44554 non-null  int64  
 1   CI_Name                     44554 non-null  object 
 2   CI_Cat                      44554 non-null  object 
 3   CI_Subcat                   44554 non-null  object 
 4   WBS                         44554 non-null  object 
 5   Incident_ID                 44554 non-null  object 
 6   Priority                    44554 non-null  float64
 7   number_cnt                  44554 non-null  float64
 8   Category                    44554 non-null  object 
 9   KB_number                   44554 non-null  object 
 10  No_of_Reassignments         44554 non-null  float64
 11  Handle_Time_hrs             44554 non-null  object 
 12  Closure_Code                44554 non-null  object 
 13  No_of_Related_Interactions  445

In [26]:
data.describe()

,Unnamed: 0,Priority,number_cnt,No_of_Reassignments,No_of_Related_Interactions,No_of_Related_Changes
count,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000,44554.000000
mean,23286.730597,4.229923,0.500118,1.152960,1.150402,1.055887
std,13473.903665,0.686527,0.288717,2.296783,2.591495,0.272356
min,0.000000,1.000000,0.000023,0.000000,1.000000,1.000000
25%,11567.500000,4.000000,0.249029,0.000000,1.000000,1.000000
50%,23311.500000,4.000000,0.501116,0.000000,1.000000,1.000000
75%,34963.750000,5.000000,0.749683,2.000000,1.000000,1.000000
max,46605.000000,5.000000,0.999997,46.000000,370.000000,9.000000


## Define X and y

In [27]:
X=data.loc[:,data.columns!='No_of_Related_Changes']
y=data.No_of_Related_Changes

In [28]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44554 entries, 0 to 46605
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  44554 non-null  int64  
 1   CI_Name                     44554 non-null  object 
 2   CI_Cat                      44554 non-null  object 
 3   CI_Subcat                   44554 non-null  object 
 4   WBS                         44554 non-null  object 
 5   Incident_ID                 44554 non-null  object 
 6   Priority                    44554 non-null  float64
 7   number_cnt                  44554 non-null  float64
 8   Category                    44554 non-null  object 
 9   KB_number                   44554 non-null  object 
 10  No_of_Reassignments         44554 non-null  float64
 11  Handle_Time_hrs             44554 non-null  object 
 12  Closure_Code                44554 non-null  object 
 13  No_of_Related_Interactions  445

## Using Label Encoder

In [29]:
enc=LabelEncoder()
X.CI_Name=enc.fit_transform(X.CI_Name)
X.CI_Cat=enc.fit_transform(X.CI_Cat)
X.Incident_ID=enc.fit_transform(X.Incident_ID)
X.CI_Subcat=enc.fit_transform(X.CI_Subcat)
X.WBS=enc.fit_transform(X.WBS)
X.Category=enc.fit_transform(X.Category)
X.KB_number=enc.fit_transform(X.KB_number)
X.Related_Interaction=enc.fit_transform(X.Related_Interaction)
X.Handle_Time_hrs=enc.fit_transform(X.Handle_Time_hrs)
X.Closure_Code=enc.fit_transform(X.Closure_Code)
X.Related_Change=enc.fit_transform(X.Related_Change)
X.head()

,Unnamed: 0,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,Related_Change
0,0,2426,11,55,135,0,4.0,0.601292,1,404,26.0,19414,6,1.0,1,2
1,1,2542,1,55,70,1,3.0,0.415050,1,447,33.0,20902,10,1.0,2,2
3,3,2542,1,55,70,2,4.0,0.642927,1,447,13.0,20818,5,1.0,3,2
4,4,2542,1,55,70,3,4.0,0.345258,1,447,2.0,17912,6,1.0,4,2
5,5,2542,1,55,70,4,4.0,0.006676,1,447,4.0,17909,6,1.0,5,2


In [30]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44554 entries, 0 to 46605
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  44554 non-null  int64  
 1   CI_Name                     44554 non-null  int32  
 2   CI_Cat                      44554 non-null  int32  
 3   CI_Subcat                   44554 non-null  int32  
 4   WBS                         44554 non-null  int32  
 5   Incident_ID                 44554 non-null  int32  
 6   Priority                    44554 non-null  float64
 7   number_cnt                  44554 non-null  float64
 8   Category                    44554 non-null  int32  
 9   KB_number                   44554 non-null  int32  
 10  No_of_Reassignments         44554 non-null  float64
 11  Handle_Time_hrs             44554 non-null  int32  
 12  Closure_Code                44554 non-null  int32  
 13  No_of_Related_Interactions  445

## Using Train-test split

In [31]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=20,test_size=0.3)

In [32]:
print("X_train shape = ",X_train.shape)
print("X_test shape = ",X_test.shape)
print("y_train shape = ",y_train.shape)
print("y_test shape = ",y_test.shape)

X_train shape =  (31187, 16)
X_test shape =  (13367, 16)
y_train shape =  (31187,)
y_test shape =  (13367,)


## Using Random-Forest Classifier algorithm
## Define the model

In [33]:
model=RandomForestClassifier(n_estimators=250,random_state=10,max_depth=4)
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [34]:
y_train_predict=model.predict(X_train)
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict)*100)

99.86534001645843


In [48]:
pd.crosstab(y_test,y_predict)

col_0,1.0,2.0,3.0,9.0
No_of_Related_Changes,,,,
1.0,12663,0,0,0
2.0,0,672,0,0
3.0,0,0,29,0
9.0,0,0,0,3


In [49]:
confusion_matrix(y_test,y_predict)

array([[12663,     0,     0,     0],
       [    0,   672,     0,     0],
       [    0,     0,    29,     0],
       [    0,     0,     0,     3]], dtype=int64)

In [37]:
print("\n Accuracy of Training = " ,accuracy_score(y_train,y_train_predict)*100)
print("\n Accuracy of Testing =  ", accuracy_score(y_test,y_predict)*100)
print("\n Precision score: {}".format(precision_score(y_test,y_predict,average='weighted')*100))
print("\n Recall score: {}".format(recall_score(y_test,y_predict,average='weighted')*100))
print("\n F1 score: {}".format(f1_score(y_test,y_predict,average='weighted')*100))


 Accuracy of Training =  99.8845672876519

 Accuracy of Testing =   99.86534001645843

 Precision score: 99.84640955500404

 Recall score: 99.86534001645843

 F1 score: 99.83543550281672


In [38]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00     12663
         2.0       0.97      1.00      0.99       672
         3.0       1.00      0.48      0.65        29
         9.0       0.00      0.00      0.00         3

    accuracy                           1.00     13367
   macro avg       0.74      0.62      0.66     13367
weighted avg       1.00      1.00      1.00     13367



## Using XGBoost

In [39]:
from xgboost import XGBClassifier

## Use train-test split

In [40]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=20,test_size=0.3)

In [41]:
print("X_train shape = ",X_train.shape)
print("X_test shape = ",X_test.shape)
print("y_train shape = ",y_train.shape)
print("y_test shape = ",y_test.shape)

X_train shape =  (31187, 16)
X_test shape =  (13367, 16)
y_train shape =  (31187,)
y_test shape =  (13367,)


## Define the model

In [42]:
model=XGBClassifier(max_depth=3,learning_rate=0.5,random_state=50,n_estimators=50)
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.5, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=50, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=50,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [43]:
y_train_predict=model.predict(X_train)
y_predict=model.predict(X_test)

In [44]:
pd.crosstab(y_test,y_predict)

col_0,1.0,2.0,3.0,9.0
No_of_Related_Changes,,,,
1.0,12663,0,0,0
2.0,0,672,0,0
3.0,0,0,29,0
9.0,0,0,0,3


In [45]:
print(confusion_matrix(y_test,y_predict))

[[12663     0     0     0]
 [    0   672     0     0]
 [    0     0    29     0]
 [    0     0     0     3]]


In [46]:
print("\nTrain accuracy = " ,accuracy_score(y_train,y_train_predict)*100)
print("\nTest accuracy = " ,accuracy_score(y_test,y_predict)*100)
print("\nPrecision score: {}".format(precision_score(y_test,y_predict,average='weighted')*100))
print("\nRecall score: {}".format(recall_score(y_test,y_predict,average='weighted')*100))
print("\nF1 score: {}".format(f1_score(y_test,y_predict,average='weighted')*100))


Train accuracy =  100.0

Test accuracy =  100.0

Precision score: 100.0

Recall score: 100.0

F1 score: 100.0


In [47]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00     12663
         2.0       1.00      1.00      1.00       672
         3.0       1.00      1.00      1.00        29
         9.0       1.00      1.00      1.00         3

    accuracy                           1.00     13367
   macro avg       1.00      1.00      1.00     13367
weighted avg       1.00      1.00      1.00     13367

